#  T M V A Classification Category
This macro provides examples for the training and testing of the
TMVA classifiers in categorisation mode.
- Project   : TMVA - a Root-integrated toolkit for multivariate data analysis
- Package   : TMVA
- Root Macro: TMVAClassificationCategory

As input data is used a toy-MC sample consisting of four Gaussian-distributed
and linearly correlated input variables with category (eta) dependent
properties.

For this example, only Fisher and Likelihood are used. Run via:

    root -l TMVAClassificationCategory.C

The output file "TMVA.root" can be analysed with the use of dedicated
macros (simply say: root -l <macro.C>), which can be conveniently
invoked through a GUI that will appear at the end of the run of this macro.



**Author:** Andreas Hoecker  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 03:46 AM.</small></i>

In [1]:
%%cpp -d
#include <cstdlib>
#include <iostream>
#include <map>
#include <string>

#include "TChain.h"
#include "TFile.h"
#include "TTree.h"
#include "TString.h"
#include "TObjString.h"
#include "TSystem.h"
#include "TROOT.h"

#include "TMVA/MethodCategory.h"
#include "TMVA/Factory.h"
#include "TMVA/DataLoader.h"
#include "TMVA/Tools.h"
#include "TMVA/TMVAGui.h"

Two types of category methods are implemented

In [2]:
Bool_t UseOffsetMethod = kTRUE;

---------------------------------------------------------------
 Example for usage of different event categories with classifiers

In [3]:
std::cout << std::endl << "==> Start TMVAClassificationCategory" << std::endl;


==> Start TMVAClassificationCategory


This loads the library

In [4]:
TMVA::Tools::Instance();

bool batchMode = false;

Create a new root output file.

In [5]:
TString outfileName( "TMVA.root" );
TFile* outputFile = TFile::Open( outfileName, "RECREATE" );

Create the factory object (see tmvaclassification.c for more information)

In [6]:
  std::string factoryOptions( "!V:!Silent:Transformations=I;D;P;G,D" );
  if (batchMode) factoryOptions += ":!Color:!DrawProgressBar";

TMVA::Factory *factory = new TMVA::Factory( "TMVAClassificationCategory", outputFile, factoryOptions );

Create dataloader

In [7]:
TMVA::DataLoader *dataloader=new TMVA::DataLoader("dataset");

Define the input variables used for the mva training

In [8]:
dataloader->AddVariable( "var1", 'F' );
dataloader->AddVariable( "var2", 'F' );
dataloader->AddVariable( "var3", 'F' );
dataloader->AddVariable( "var4", 'F' );

You can add so-called "spectator variables", which are not used in the mva training,
 but will appear in the final "TestTree" produced by TMVA. This TestTree will contain the
 input variables, the response values of all trained MVAs, and the spectator variables

In [9]:
dataloader->AddSpectator( "eta" );

Load the signal and background event samples from root trees

In [10]:
TFile *input(0);
TString fname = TString(gSystem->DirName(__FILE__) ) + "/data/";
if (gSystem->AccessPathName( fname + "toy_sigbkg_categ_offset.root")) {
   // if directory data not found try using tutorials dir
   fname = gROOT->GetTutorialDir() + "/tmva/data/";
}
if (UseOffsetMethod) fname += "toy_sigbkg_categ_offset.root";
else                 fname += "toy_sigbkg_categ_varoff.root";
if (!gSystem->AccessPathName( fname )) {
   // first we try to find tmva_example.root in the local directory
   std::cout << "--- TMVAClassificationCategory: Accessing " << fname << std::endl;
   input = TFile::Open( fname );
}

if (!input) {
   std::cout << "ERROR: could not open data file: " << fname << std::endl;
   exit(1);
}

TTree *signalTree     = (TTree*)input->Get("TreeS");
TTree *background = (TTree*)input->Get("TreeB");

--- TMVAClassificationCategory: Accessing /mnt/build/workspace/root-makedoc-v618/rootspi/rdoc/src/v6-18-00-patches.build/tutorials/tmva/data/toy_sigbkg_categ_offset.root


Global event weights per tree (see below for setting event-wise weights)

In [11]:
Double_t signalWeight     = 1.0;
Double_t backgroundWeight = 1.0;

You can add an arbitrary number of signal or background trees

In [12]:
dataloader->AddSignalTree    ( signalTree,     signalWeight     );
dataloader->AddBackgroundTree( background, backgroundWeight );

<HEADER> DataSetInfo              : [dataset] : Added class "Signal"
                         : Add Tree TreeS of type Signal with 10000 events
<HEADER> DataSetInfo              : [dataset] : Added class "Background"
                         : Add Tree TreeB of type Background with 10000 events


Apply additional cuts on the signal and background samples (can be different)

In [13]:
TCut mycuts = ""; // for example: TCut mycuts = "abs(var1)<0.5 && abs(var2-0.5)<1";
TCut mycutb = ""; // for example: TCut mycutb = "abs(var1)<0.5";

Tell the factory how to use the training and testing events

In [14]:
dataloader->PrepareTrainingAndTestTree( mycuts, mycutb,
                                     "nTrain_Signal=0:nTrain_Background=0:SplitMode=Random:NormMode=NumEvents:!V" );

Book mva methods

Fisher discriminant

In [15]:
factory->BookMethod( dataloader, TMVA::Types::kFisher, "Fisher", "!H:!V:Fisher" );

<HEADER> Factory                  : Booking method: Fisher
                         : 


Likelihood

In [16]:
factory->BookMethod( dataloader, TMVA::Types::kLikelihood, "Likelihood",
                     "!H:!V:TransformOutput:PDFInterpol=Spline2:NSmoothSig[0]=20:NSmoothBkg[0]=20:NSmoothBkg[1]=10:NSmooth=1:NAvEvtPerBin=50" );

<HEADER> Factory                  : Booking method: Likelihood
                         : 


Categorised classifier

In [17]:
TMVA::MethodCategory* mcat = 0;

The variable sets

In [18]:
TString theCat1Vars = "var1:var2:var3:var4";
TString theCat2Vars = (UseOffsetMethod ? "var1:var2:var3:var4" : "var1:var2:var3");

Fisher with categories

In [19]:
TMVA::MethodBase* fiCat = factory->BookMethod( dataloader, TMVA::Types::kCategory, "FisherCat","" );
mcat = dynamic_cast<TMVA::MethodCategory*>(fiCat);
mcat->AddMethod( "abs(eta)<=1.3", theCat1Vars, TMVA::Types::kFisher, "Category_Fisher_1","!H:!V:Fisher" );
mcat->AddMethod( "abs(eta)>1.3",  theCat2Vars, TMVA::Types::kFisher, "Category_Fisher_2","!H:!V:Fisher" );

<HEADER> Factory                  : Booking method: FisherCat
                         : 
                         : Adding sub-classifier: Fisher::Category_Fisher_1
<HEADER> DataSetInfo              : [Category_Fisher_1_dsi] : Added class "Signal"
<HEADER> DataSetInfo              : [Category_Fisher_1_dsi] : Added class "Background"
                         : Adding sub-classifier: Fisher::Category_Fisher_2
<HEADER> DataSetInfo              : [Category_Fisher_2_dsi] : Added class "Signal"
<HEADER> DataSetInfo              : [Category_Fisher_2_dsi] : Added class "Background"


Likelihood with categories

In [20]:
TMVA::MethodBase* liCat = factory->BookMethod( dataloader, TMVA::Types::kCategory, "LikelihoodCat","" );
mcat = dynamic_cast<TMVA::MethodCategory*>(liCat);
mcat->AddMethod( "abs(eta)<=1.3",theCat1Vars, TMVA::Types::kLikelihood,
                 "Category_Likelihood_1","!H:!V:TransformOutput:PDFInterpol=Spline2:NSmoothSig[0]=20:NSmoothBkg[0]=20:NSmoothBkg[1]=10:NSmooth=1:NAvEvtPerBin=50" );
mcat->AddMethod( "abs(eta)>1.3", theCat2Vars, TMVA::Types::kLikelihood,
                 "Category_Likelihood_2","!H:!V:TransformOutput:PDFInterpol=Spline2:NSmoothSig[0]=20:NSmoothBkg[0]=20:NSmoothBkg[1]=10:NSmooth=1:NAvEvtPerBin=50" );

<HEADER> Factory                  : Booking method: LikelihoodCat
                         : 
                         : Adding sub-classifier: Likelihood::Category_Likelihood_1
<HEADER> DataSetInfo              : [Category_Likelihood_1_dsi] : Added class "Signal"
<HEADER> DataSetInfo              : [Category_Likelihood_1_dsi] : Added class "Background"
                         : Adding sub-classifier: Likelihood::Category_Likelihood_2
<HEADER> DataSetInfo              : [Category_Likelihood_2_dsi] : Added class "Signal"
<HEADER> DataSetInfo              : [Category_Likelihood_2_dsi] : Added class "Background"


Now you can tell the factory to train, test, and evaluate the mvas

Train mvas using the set of training events

In [21]:
factory->TrainAllMethods();

<HEADER> Factory                  : Train all methods
<HEADER> DataSetFactory           : [dataset] : Number of events in input trees
                         : 
                         : 
                         : Number of training and testing events
                         : ---------------------------------------------------------------------------
                         : Signal     -- training events            : 5000
                         : Signal     -- testing events             : 5000
                         : Signal     -- training and testing events: 10000
                         : Background -- training events            : 5000
                         : Background -- testing events             : 5000
                         : Background -- training and testing events: 10000
                         : 
<HEADER> DataSetInfo              : Correlation matrix (Signal):
                         : ----------------------------------------
                         :   

0%, time left: unknown
7%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
38%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
57%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time left: 0 sec
88%, time left: 0 sec
94%, time left: 0 sec
0%, time left: unknown
7%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
38%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
57%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time left: 0 sec
88%, time left: 0 sec


                         : Elapsed time for evaluation of 10000 events: 0.00806 sec       
                         : Creating xml weight file: dataset/weights/TMVAClassificationCategory_Likelihood.weights.xml
                         : Creating standalone class: dataset/weights/TMVAClassificationCategory_Likelihood.class.C
                         : TMVA.root:/dataset/Method_Likelihood/Likelihood


94%, time left: 0 sec


<HEADER> Factory                  : Training finished
                         : 
<HEADER> Factory                  : Train method: FisherCat for Classification
                         : 
                         : Train all sub-classifiers for Classification ...
<HEADER> DataSetFactory           : [Category_Fisher_1_dsi] : Number of events in input trees
                         : Dataset[Category_Fisher_1_dsi] :     Signal     requirement: "abs(eta)<=1.3"
                         : Dataset[Category_Fisher_1_dsi] :     Signal          -- number of events passed: 5123   / sum of weights: 5123 
                         : Dataset[Category_Fisher_1_dsi] :     Signal          -- efficiency             : 0.5123
                         : Dataset[Category_Fisher_1_dsi] :     Background requirement: "abs(eta)<=1.3"
                         : Dataset[Category_Fisher_1_dsi] :     Background      -- number of events passed: 5134   / sum of weights: 5134 
                         : Dataset[Categ

...

0%, time left: unknown
6%, time left: 0 sec
12%, time left: 0 sec
18%, time left: 0 sec
25%, time left: 0 sec
31%, time left: 0 sec
37%, time left: 0 sec
43%, time left: 0 sec
50%, time left: 0 sec
56%, time left: 0 sec
62%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
81%, time left: 0 sec
87%, time left: 0 sec
94%, time left: 0 sec
0%, time left: unknown
6%, time left: 0 sec
12%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
31%, time left: 0 sec
38%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
57%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
81%, time left: 0 sec
87%, time left: 0 sec
94%, time left: 0 sec


                         : Training finished
                         : Begin ranking of input variables...
<HEADER> Category_Likelihood_1    : Ranking result (top variable is best ranked)
                         : -----------------------------------
                         : Rank : Variable  : Delta Separation
                         : -----------------------------------
                         :    1 : var4      : 1.031e-01
                         :    2 : var3      : 1.716e-02
                         :    3 : var1      : 1.036e-02
                         :    4 : var2      : 4.428e-03
                         : -----------------------------------
<HEADER> Category_Likelihood_2    : Ranking result (top variable is best ranked)
                         : -----------------------------------
                         : Rank : Variable  : Delta Separation
                         : -----------------------------------
                         :    1 : var4      : 1.424e-01
         

0%, time left: unknown
7%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
38%, time left: 0 sec
44%, time left: 0 sec


                         : Elapsed time for evaluation of 10000 events: 0.0116 sec       
                         : Creating xml weight file: dataset/weights/TMVAClassificationCategory_LikelihoodCat.weights.xml
<HEADER> Factory                  : Training finished
                         : 
                         : Ranking input variables (method specific)...
<HEADER> Fisher                   : Ranking result (top variable is best ranked)
                         : -------------------------------
                         : Rank : Variable  : Discr. power
                         : -------------------------------
                         :    1 : var4      : 1.446e-01
                         :    2 : var3      : 7.153e-02
                         :    3 : var2      : 2.447e-02
                         :    4 : var1      : 1.243e-02
                         : -------------------------------
<HEADER> Likelihood               : Ranking result (top variable is best ranked)
            

50%, time left: 0 sec
57%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time left: 0 sec
88%, time left: 0 sec
94%, time left: 0 sec


Evaluate all mvas using the set of test events

In [22]:
factory->TestAllMethods();

<HEADER> Factory                  : Test all methods
<HEADER> Factory                  : Test method: Fisher for Classification performance
                         : 
<HEADER> Fisher                   : [dataset] : Evaluation of Fisher on testing sample (10000 events)
                         : Elapsed time for evaluation of 10000 events: 0.00149 sec       
<HEADER> Factory                  : Test method: Likelihood for Classification performance
                         : 
<HEADER> Likelihood               : [dataset] : Evaluation of Likelihood on testing sample (10000 events)
                         : Elapsed time for evaluation of 10000 events: 0.00654 sec       
<HEADER> Factory                  : Test method: FisherCat for Classification performance
                         : 
<HEADER> FisherCat                : [dataset] : Evaluation of FisherCat on testing sample (10000 events)
                         : Elapsed time for evaluation of 10000 events: 0.00235 sec       
<HEADER> 

0%, time left: unknown
7%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
38%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
57%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time left: 0 sec
88%, time left: 0 sec
94%, time left: 0 sec
0%, time left: unknown
7%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
38%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
57%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time left: 0 sec
88%, time left: 0 sec
94%, time left: 0 sec
0%, time left: unknown
7%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
38%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
57%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time 

Evaluate and compare performance of all configured mvas

In [23]:
factory->EvaluateAllMethods();

<HEADER> Factory                  : Evaluate all methods
<HEADER> Factory                  : Evaluate classifier: Fisher
                         : 
<HEADER> Fisher                   : [dataset] : Loop over test events and fill histograms with classifier response...
                         : 
<HEADER> TFHandler_Fisher         : Variable        Mean        RMS   [        Min        Max ]
                         : -----------------------------------------------------------
                         :     var1:  -0.014081     1.2910   [    -5.3119     4.5609 ]
                         :     var2:  -0.014399     1.3299   [    -4.7537     4.6723 ]
                         :     var3:  -0.027971     1.3779   [    -5.2892     4.7007 ]
                         :     var4:    0.12966     1.4883   [    -5.1002     4.9767 ]
                         : -----------------------------------------------------------
<HEADER> Factory                  : Evaluate classifier: Likelihood
                   

--------------------------------------------------------------

Save the output

In [24]:
outputFile->Close();

std::cout << "==> Wrote root file: " << outputFile->GetName() << std::endl;
std::cout << "==> TMVAClassificationCategory is done!" << std::endl;

==> Wrote root file: TMVA.root
==> TMVAClassificationCategory is done!


Clean up

In [25]:
delete factory;
delete dataloader;

Launch the gui for the root macros

In [26]:
if (!gROOT->IsBatch()) TMVA::TMVAGui( outfileName );